### **Speech Recognition**

In [43]:
!pip3 install SpeechRecognition
!pip3 install pyaudio
import speech_recognition as sr

In [44]:
# create a Recognizer instance
r = sr.Recognizer()

In [45]:
harvard = sr.AudioFile('audio_files/harvard.wav')
with harvard as source:
    audio = r.record(source)
r.recognize_google(audio)

'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite a zestful food is be hot cross bun'

In [46]:
# offset: starting point
with harvard as source:
    audio = r.record(source, offset=4, duration=3)
r.recognize_google(audio)

'it takes heat to bring out the odor'

In [47]:
jackhammer = sr.AudioFile('audio_files/jackhammer.wav')
with jackhammer as source:
    audio = r.record(source)
r.recognize_google(audio)

'the snail smelly old gear vendors'

In [48]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)
r.recognize_google(audio)

'Bill smell like old beer drinkers'

In [49]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source, duration=0.5)
    audio = r.record(source)
r.recognize_google(audio)

'the stale smell of old gear vendors'

In [50]:
r.recognize_google(audio, show_all=True)

{'alternative': [{'transcript': 'the stale smell of old gear vendors'},
  {'transcript': 'does still smell old gear vendors'},
  {'transcript': 'the smell smell of old gear vendors'},
  {'transcript': 'the snail smell of old gear vendors'},
  {'transcript': 'the still smell of old gear vendors'},
  {'transcript': 'the snail smelly old gear vendors'},
  {'transcript': 'the still smell old gear vendors'},
  {'transcript': 'does still sell old gear vendors'},
  {'transcript': 'does still smell old gear winters'}],
 'final': True}

#### **Working With Microphones**

In [51]:
mic = sr.Microphone()

In [52]:
sr.Microphone.list_microphone_names()

['HP Z27',
 'Built-in Microphone',
 'Built-in Output',
 'Eric’s AirPods Pro',
 'Eric’s AirPods Pro']

In [53]:
mic = sr.Microphone(device_index=3)

In [54]:
with mic as source:
    audio = r.listen(source)

In [55]:
r.recognize_google(audio, language="zh")

'哈喽'

#### **A "Guess the Word" Game**

In [18]:
import time
import random
import speech_recognition as sr

In [20]:
def recognize_speech_from_mic(recognizer, microphone):
    
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")
    
    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["error"] = "Unable to recognize speech"

    return response

In [28]:
WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
NUM_GUESSES = 3
PROMPT_LIMIT = 5

recognizer = sr.Recognizer()
microphone = sr.Microphone()

word = random.choice(WORDS)

instructions = (
    "I'm thinking of one of these words:\n"
    "{words}\n"
    "You have {n} tries to guess which one.\n"
).format(words=', '.join(WORDS), n=NUM_GUESSES)

# show instructions and wait 3 seconds before starting the game
print(instructions)
time.sleep(3)

for i in range(NUM_GUESSES):
    for j in range(PROMPT_LIMIT):
        print('Guess {}. Speak!'.format(i+1))
        guess = recognize_speech_from_mic(recognizer, microphone)
        if guess["transcription"]:
            break
        if not guess["success"]:
            break
        print("I didn't catch that. What did you say?\n")

    if guess["error"]:
        print("ERROR: {}".format(guess["error"]))
        break

    print("You said: {}".format(guess["transcription"]))

    guess_is_correct = guess["transcription"].lower() == word.lower()
    user_has_more_attempts = i < NUM_GUESSES - 1

    if guess_is_correct:
        print("Correct! You win!".format(word))
        break
    elif user_has_more_attempts:
        print("Incorrect. Try again.\n")
    else:
        print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
        break

I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak!
I didn't catch that. What did you say?

Guess 1. Speak!
I didn't catch that. What did you say?

Guess 1. Speak!
I didn't catch that. What did you say?

Guess 1. Speak!
I didn't catch that. What did you say?

Guess 1. Speak!
I didn't catch that. What did you say?

ERROR: Unable to recognize speech
